In [24]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from pprint import pprint
import shutil

In [25]:
def get_all_image_input(directory):
    list_image = glob.glob(directory + "*.jpg")
    list_image.extend(glob.glob(directory + "*.png"))
    list_image.extend(glob.glob(directory + "*.jpeg"))
    list_image.extend(glob.glob(directory + "*.PNG"))
    return list_image

In [26]:
def show_image(img):
    image=cv2.imread(img)
    plt.imshow(image)
    plt.title("Image")
    plt.show()

In [52]:
def get_contours(img_bin, dir):
    # Các đường kẻ ngang + dọc
    kernel_length = np.array(img_bin).shape[1] // 35

    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    img_temp1 = cv2.erode(img_bin, ver_kernel, iterations = 3)
    vertical_lines_img = cv2.dilate(img_temp1, ver_kernel, iterations=3)
    cv2.imwrite(dir+"/vertical_lines_img.png", vertical_lines_img)

    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
    cv2.imwrite(dir+"/horizontal_lines_img.png", horizontal_lines_img)


    # matrix 3x3
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

    # img_final_bin = vertical_lines_img + horizontal_lines_img
    img_final_bin = cv2.addWeighted(vertical_lines_img, 0.5, horizontal_lines_img, 0.5, 0.0)
    img_final_bin = 255 - img_final_bin
    img_final_bin = cv2.erode(img_final_bin, kernel, iterations=2)
    cv2.imwrite(dir+"/img_final_bin.png", img_final_bin)

    (thresh, img_final) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Tìm các contour của ảnh
    im, contours, hierarchy = cv2.findContours(img_final, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return (contours, img_final)

In [53]:
def get_boxes(img, dir):
    height, width = img.shape[:2]
        
    (thresh, img_bin) = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)

    img_bin = 255 - img_bin

    (contours, img_final) = get_contours(img_bin, dir)

    contours.reverse()
    boxes=[]
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x != 0 and y !=0 and w != width and h != height:
            cv2.rectangle(img_final, (x, y), (x + w, y + h), (0, 0, 0), 1)
            boxes.append([x,y,w,h])

    cv2.imwrite(dir + "/img_final.png", img_final)
    return boxes

In [54]:
def find_table_box(list_boxes, img_height):
    x_es=[]
    for box in list_boxes:
        x_es.append(box[0])
    
    results=[]
    min_height=0
    while(len(x_es) != 0):
        min_x = min(x_es)
        x_es.remove(min_x)
        min_y = img_height
        
        res = []
        for box in list_boxes:
            if min_x in box and box[1] < min_y and (box[1] + box[3]) > min_height:
                res = box
                min_y = box[1]
                min_height = (box[1] + box[3])
                
        for box in list_boxes:
            if box == res:
                results.append(res)
                list_boxes.remove(box)
            
    return results

In [55]:
def main(list_image):
    for image in list_image:
        print("\nProcessing", image, "---------")
        
        # Tạo thư mục output
        basename = os.path.basename(image)
        filename = os.path.splitext(basename)[0]
        dir = './output/' + filename
        if os.path.exists(dir):
            shutil.rmtree(dir)
        os.mkdir(dir)
            
        img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
        height, width = img.shape[:2]
        
        boxes = get_boxes(img, dir)
        
        # ----------------
        table = find_table_box(boxes, height)
        
        for t in table:
            x,y,w,h = t[:]
            img_table = img[y:y+h, x:x+w]
            h, w = img_table.shape[:2]
            
            table_boxes=get_boxes(img_table, dir+"/table")
            print("Detect table", str(table.index(t)))
            if detect_table(table_boxes, h, w):
                cv2.imwrite(dir + "/_img" + str(table.index(t)) + ".png", img_table)
                print("\tSaved", filename + "/_img"+str(table.index(t))+".png", img_table.shape[:])

In [56]:
def detect_table(boxes, h, w):
    y_es=[box[1] for box in boxes]
    y_es=list(set(y_es))
    y_es=sorted(y_es)
    
    rows=[]
    minH=h
    minW=w
    for y in y_es:
        row=[]
        for box in boxes:
            if box[1]==y and box[2]>5 and box[3]>5:
                row.append(box)
        if row:
            rows.append(row)
    
    pprint(rows)
    
    if rows:
        if len(rows) == 1:
            return False
        else:
            return True
    else:
        return False

In [67]:
input = get_all_image_input('./input/')
pprint(input)

['./input/paper - Copy.jpg',
 './input/paper.jpg',
 './input/4.png',
 './input/Capture.png',
 './input/3.png',
 './input/table - Copy.png',
 './input/test.png',
 './input/table.png',
 './input/Hinh.png',
 './input/1.png',
 './input/2.png',
 './input/4.PNG',
 './input/5.PNG']


In [68]:
main(input)


Processing ./input/paper - Copy.jpg ---------
Detect table 0
[[[11, 11, 112, 54],
  [133, 11, 112, 54],
  [255, 11, 108, 54],
  [373, 11, 110, 54]],
 [[11, 75, 112, 20],
  [133, 75, 112, 20],
  [255, 75, 108, 20],
  [373, 75, 110, 20]],
 [[11, 105, 112, 21],
  [133, 105, 112, 21],
  [255, 105, 108, 21],
  [373, 105, 110, 21]],
 [[11, 136, 112, 23],
  [133, 136, 112, 23],
  [255, 136, 108, 23],
  [373, 136, 110, 23]]]
	Saved paper - Copy/_img0.png (170, 494)
Detect table 1
[[[11, 11, 113, 50],
  [134, 11, 107, 50],
  [251, 11, 108, 50],
  [369, 11, 114, 50]],
 [[11, 71, 113, 21],
  [134, 71, 107, 21],
  [251, 71, 108, 21],
  [369, 71, 114, 21]],
 [[11, 102, 113, 20],
  [134, 102, 107, 20],
  [251, 102, 108, 20],
  [369, 102, 114, 20]],
 [[11, 132, 113, 21],
  [134, 132, 107, 21],
  [251, 132, 108, 21],
  [369, 132, 114, 21]],
 [[11, 163, 472, 21]]]
	Saved paper - Copy/_img1.png (195, 494)

Processing ./input/paper.jpg ---------
Detect table 0
[[[11, 11, 112, 54],
  [133, 11, 112, 54],


In [ ]:
#             (thresh, img_bin) = cv2.threshold(img_table, 200, 255, cv2.THRESH_BINARY)
#             img_bin = 255 - img_bin
        
#             (_im, _contours, _hierarchy, _img) = get_contours(img_bin)
# #             contours.reverse()
#             _height, _width = _img.shape[:2]
#             for contour in _contours:
#                 x, y, w, h = cv2.boundingRect(contour)
#                 if w > h and (w/_width)<0.6 and (h/_height)<0.6:
#                     cv2.rectangle(_img, (x, y), (x + w, y + h), (0, 0, 0), 2)
            
#             print("Locate:",x, y, x+w, y+h)

In [115]:
ar=[[122, 667, 113, 50],
 [245, 667, 107, 50],
 [362, 667, 108, 50],
 [480, 667, 114, 50],
 [122, 727, 113, 21],
 [245, 727, 107, 21],
 [362, 727, 108, 21],
 [480, 727, 114, 21],
 [122, 758, 113, 20],
 [245, 758, 107, 20],
 [362, 758, 108, 20],
 [480, 758, 114, 20],
 [122, 788, 113, 21],
 [245, 788, 107, 21],
 [362, 788, 108, 21],
 [480, 788, 114, 21],
 [122, 819, 472, 21]]
pprint(ar)

[[122, 667, 113, 50],
 [245, 667, 107, 50],
 [362, 667, 108, 50],
 [480, 667, 114, 50],
 [122, 727, 113, 21],
 [245, 727, 107, 21],
 [362, 727, 108, 21],
 [480, 727, 114, 21],
 [122, 758, 113, 20],
 [245, 758, 107, 20],
 [362, 758, 108, 20],
 [480, 758, 114, 20],
 [122, 788, 113, 21],
 [245, 788, 107, 21],
 [362, 788, 108, 21],
 [480, 788, 114, 21],
 [122, 819, 472, 21]]
